In [2]:
from ltp import LTP
ltp = LTP()

In [5]:
import string
from zhon.hanzi import punctuation
import re
from ltp import LTP
ltp = LTP()

class Custom_segment:
    def __init__(self, add_words=None):
        if add_words:
            ltp.add_words(words=add_words, max_window=max([len(w) for w in add_words]))
        self.stopwords = self.get_stopwords()

    def get_stopwords(self):
        with open('../text_process/baidu_stopwords.txt','r', encoding='utf-8') as f:
            content = f.readlines()
            stopwords = list(map(str.strip, content))
        return  stopwords

    def cut_sentence(self, text):
        sents = ltp.sent_split([text])
        return sents

    def tokenizers(self, sents):
        segs = ltp.seg(sents)[0]
        rem_p_segments = map(self.remove_punctuation, segs)
        rem_sw_segments = map(self.del_stopwords, rem_p_segments)
        return list(rem_sw_segments)

    def remove_punctuation(self, words):
        new_words = [word for word in words if word not in punctuation and word not in string.punctuation]
        return new_words     

    def del_stopwords(self, words):
        new_words = [word for word in words if word not in self.stopwords]
        return new_words
        
    def __call__(self, text, rm_p=False, del_s=False):

        sents = self.cut_sentence(text)
        segments = self.tokenizers(sents)
        return sents, segments


if __name__=="__main__":
    text = "2017年11月21日，全国中小企业股份转让系统有限责任公司出具《关于同意江苏鸿基节能新技术股份有限公司终止股票全国中小企业股份转让系统挂牌的函》“股转系统函 [2017]6631号”，\
        同意鸿基节能股票自2017年11月24日在全国中小企业股份转让系统终止挂牌。同日，鸿基节能出具《关于江苏鸿基节能新技术股份有限公司股票终止挂牌的公告》，\
        鸿基节能决定自2017年11月24日起终止其股票挂牌"
    custom_segment = Custom_segment()
    sents, segments = custom_segment(text)
    print(f'sents:{sents}, segments:{segments}')
    segments = custom_segment.tokenizers(['建筑工程施工环境安全状态智能监测及安全监控装置研究'])
    print(segments)

sents:['2017年11月21日，全国中小企业股份转让系统有限责任公司出具《关于同意江苏鸿基节能新技术股份有限公司终止股票全国中小企业股份转让系统挂牌的函》“股转系统函 [2017]6631号”，        同意鸿基节能股票自2017年11月24日在全国中小企业股份转让系统终止挂牌。', '同日，鸿基节能出具《关于江苏鸿基节能新技术股份有限公司股票终止挂牌的公告》，        鸿基节能决定自2017年11月24日起终止其股票挂牌'], segments:[['2017年', '11月', '21日', '全国', '中小企业', '股份转让', '系统', '有限', '责任', '公司', '出具', '同意', '江苏', '鸿基', '节能', '新技术', '股份', '有限公司', '终止', '股票', '全国', '中小企业', '股份转让', '系统', '挂牌', '函', '股转', '系统函', '2017]6631号', '同意', '鸿基', '节能', '股票', '2017年', '11月', '24日', '全国', '中小企业', '股份', '转让', '系统', '终止', '挂牌'], ['同日', '鸿基', '节能', '出具', '江苏', '鸿基', '节能', '新', '技术', '股份', '有限公司', '股票', '终止', '挂牌', '公告', '鸿基', '节能', '2017年', '11月', '24日', '终止', '股票', '挂牌']]
[['建筑', '工程', '施工', '环境', '状态', '智能', '监测', '监控', '装置', '研究']]


In [10]:
import random
import re

class Lexical_analysis:

    @staticmethod
    def inclusion_rate(left_segment, right_segment, left_sentence, right_sentence):

        '''
        Calculate the inclusion degree of the intersection of left segment and right segment in right segment

        Parameters
        ----------
        input: left_segment, right_segment,left_sentence, right_sentence
        Returns
        ----------
        rate: float
        '''
        if right_sentence in left_sentence:
            rate = 1.1
        else:
            intersection = set(left_segment) & set(right_segment)
            # print(intersection)
            rate = len(intersection) / len(set(right_segment))
        return rate

    @staticmethod
    def get_search_str(left_segment, right_segment, left_sentence, right_sentence):
        '''
        find the serach string that use for add footnote in left_sentence 

        Parameters
        ----------
        input: left_sentence, right_sentence， left_segment, right_segment
        Returns
        ----------
        search_str: string
        '''    
        # Full character matching
        if right_sentence in left_sentence:
            if re.findall(right_sentence + '(.)', left_sentence) and re.findall(right_sentence + '(.)', left_sentence)[0] in '》”’）'\
                and not right_sentence.endswith('》') :
                search_str = re.findall(right_sentence + '.', left_sentence)[0] 
            else:
                search_str = right_sentence
        else:
            # Calculate the intersection of left_segment and right_segment
            intersection = [seg for seg in right_segment if seg in left_segment]

            # Remove duplicates and preserving order of elements
            unique_intersection_r = sorted(list(dict.fromkeys(intersection).keys()),key=right_segment.index)
            unique_intersection_l = sorted(list(dict.fromkeys(intersection).keys()),key=left_segment.index)

            # Pick the words as matching words
            matching_words_r = sorted(unique_intersection_r, key=right_segment.index)
            matching_words_l = sorted(unique_intersection_l, key=left_segment.index)
            # search in matching_words right sentence
            pattern_r = ''
            for mwr in matching_words_r:
                pattern_r = pattern_r + mwr + '.*?'
            print(f'pattern_r:{pattern_r}' ) #
            # search in matching_words left sentence
            pattern_l = ''
            for mwl in matching_words_l:
                pattern_l = pattern_l + mwl + '.*?'                              
            f'pattern_l:{pattern_l}' #
            if re.findall(pattern_r + '(.)', left_sentence) and \
                re.findall(pattern_r + '(.)', left_sentence)[0] in '》”’）':
                search_str = re.findall(pattern_r + '.', left_sentence)[0]
            elif re.search(pattern_r + '：“.+?”', left_sentence):
                search_str = re.search(pattern_r + '：“.+?”', left_sentence).group()
            elif re.findall(pattern_r, left_sentence):
                search_str = re.findall(pattern_r, left_sentence)[0]
            elif re.findall(pattern_l + '(.)', left_sentence) and \
                re.findall(pattern_l + '(.)', left_sentence)[0] in '》”’）':
                search_str = re.findall(pattern_l + '.', left_sentence)[0]
            elif re.search(pattern_l + '：“.+?”', left_sentence):
                search_str = re.search(pattern_l + '：“.?”', left_sentence).group()
            else :
                search_str = re.findall(pattern_l, left_sentence)[0] 
        return search_str 

class Dependency_Parser:
    pass

if __name__=="__main__":

    custom_segment = Custom_segment()
    left_sentence = '2017年11月21日，全国中小企业股份转让系统有限责任公司出具《关于同意江苏鸿基节能新技术股份有限公司终止股票全国中小企业股份转让系统挂牌的函》“股转系统函 [2017]6631号”，同意鸿基节能股票自2017年11月24日在全国中小企业股份转让系统终止挂牌。同日，鸿基节能出具《关于江苏鸿基节能新技术股份有限公司股票终止挂牌的公告》，鸿基节能决定自2017年11月24日起终止其股票挂牌'
    right_sentence = "关于同意江苏鸿基节能新技术股份有限公司终止股票全国中小企业股份转让系统挂牌的函"
    right_sentence2 = "《关于同意江苏鸿基节能新技术股份有限公司终止股票全国中小企业股份转让系统挂牌的函》（股转系统函[2017]6631号）"

    left_segment = custom_segment.tokenizers([left_sentence])[0]
    right_segment = custom_segment.tokenizers([right_sentence])[0]
    right_segment2 = custom_segment.tokenizers([right_sentence2])[0]
    print(f'left_segment: {left_segment}')
    print(f'right_segment: {right_segment}')
    print(f'right_segment2: {right_segment2}')

    rate = Lexical_analysis.inclusion_rate(left_segment, right_segment, left_sentence, right_sentence)
    rate2 = Lexical_analysis.inclusion_rate(left_segment, right_segment, left_sentence, right_sentence2)
    print(f'rate: {rate}')
    print(f'rate2: {rate2}')


    search_str = Lexical_analysis.get_search_str(left_segment, right_segment, left_sentence, right_sentence)
    print(f'search_str: {search_str}')

    search_str2 = Lexical_analysis.get_search_str(left_segment, right_segment2, left_sentence, right_sentence2) 
    print(f'search_str2: {search_str2}')

left_segment: ['2017年', '11月', '21日', '全国', '中小企业', '股份', '转让', '系统', '有限', '责任', '公司', '出具', '同意', '江苏', '鸿基', '节能', '新', '技术', '股份', '有限公司', '终止', '股票', '全国', '中小企业', '股份', '转让', '系统', '挂牌', '函', '股转', '系统', '函', '2017', '6631', '号', '同意', '鸿基', '节能', '股票', '2017年', '11月', '24日', '全国', '中小企业', '股份', '转让', '系统', '终止', '挂牌', '同日', '鸿基', '节能', '出具', '江苏', '鸿基', '节能', '新', '技术', '股份', '有限公司', '股票', '终止', '挂牌', '公告', '鸿基', '节能', '2017年', '11月', '24日', '终止', '股票', '挂牌']
right_segment: ['同意', '江苏', '鸿基', '节能', '新', '技术', '股份', '有限公司', '终止', '股票', '全国', '中小企业', '股份', '转让', '系统', '挂牌', '函']
right_segment2: ['同意', '江苏', '鸿基', '节能', '新', '技术', '股份', '有限公司', '终止', '股票', '全国', '中小企业', '股份', '转让', '系统', '挂牌', '函', '股转', '系统', '函', '2017', '6631', '号']
rate: 1.1
rate2: 1.0
search_str: 关于同意江苏鸿基节能新技术股份有限公司终止股票全国中小企业股份转让系统挂牌的函》
pattern_r:同意.*?江苏.*?鸿基.*?节能.*?新.*?技术.*?股份.*?有限公司.*?终止.*?股票.*?全国.*?中小企业.*?转让.*?系统.*?挂牌.*?函.*?股转.*?2017.*?6631.*?号.*?
search_str2: 同意江苏鸿基节能新技术股份有限公司终止股票全国中小企业股份转让系统挂牌的函》“股转系统函 [2